In [25]:
# Handle table-like data and matrices
import numpy as np
import pandas as pd
import tqdm
import os
import time
import pickle


from collections import Counter

# Modelling Algorithms
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC, LinearSVC
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# Modelling Helpers
from sklearn.preprocessing import Normalizer , scale, LabelEncoder , StandardScaler ,MinMaxScaler
from sklearn.feature_selection import RFECV
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, StratifiedKFold, learning_curve , KFold
from sklearn.metrics import r2_score, mean_squared_error,confusion_matrix,classification_report,accuracy_score
from sklearn.utils import shuffle

# Visualisation
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns

# Configure visualisations
%matplotlib inline
plt.rcParams['figure.figsize'] = [14,14]
mpl.style.use( 'ggplot' )
sns.set_style( 'whitegrid' )
pylab.rcParams[ 'figure.figsize' ] = 12 , 8

In [26]:
%macro -q __impC 25

In [27]:
%store __impC

Stored '__impC' (Macro)


In [1]:
# Handle table-like data and matrices
import numpy as np
import pandas as pd
import tqdm
import os
import time
import pickle


from collections import Counter

# Modelling Algorithms
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR, LinearSVR
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

# Modelling Helpers
from sklearn.preprocessing import Normalizer , scale,StandardScaler,MinMaxScaler
from sklearn.feature_selection import RFECV
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, KFold, learning_curve
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.utils import shuffle

# Visualisation
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns

# Configure visualisations
%matplotlib inline
plt.rcParams['figure.figsize'] = [14,14]
mpl.style.use( 'ggplot' )
sns.set_style( 'whitegrid' )
pylab.rcParams[ 'figure.figsize' ] = 12 , 8

C:\Users\Rohit\anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\Rohit\anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\Rohit\anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\Rohit\anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\Rohit\anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 19

In [2]:
%macro -q __impR 1

In [3]:
%store __impR

Stored '__impR' (Macro)


In [28]:
class SNKFC:
    
    def __init__(self, models, k,s):
        self.models = models
        self.k = k
        self.s = s
    
    def fit(self,data,target):
        kf = KFold(n_splits=self.k)
        n1 = len(self.models)
        L=0
        for model in self.models:
            n = len(model)
            i=0
            layer_accuracy = []
            pred = pd.DataFrame()
            for train_index , val_index in kf.split(data):
                X_train , X_val = data.loc[train_index],data.loc[val_index]
                y_train , y_val = target[train_index],target[val_index]
                c= 0
                while c < n:
                    model[c].fit(X_train,y_train)
                    pred[i] = model[c].predict(X_val)
                    c=c+1
                    i=i+1
            for t in np.arange(kf.n_splits):
                if t == 0:
                    X = pred[np.arange(0,n)]
                else:
                    Y = pred[np.arange(t*n,(1+t)*n)].set_index(np.arange(t*int(data.shape[0]/kf.n_splits),(t+1)*int(data.shape[0]/kf.n_splits)))
                    for g in np.arange(n):
                        Y[g]=Y[(n*t)+g]
                        Y = Y.drop(n*t+g,axis=1)
                    X = pd.concat([X,Y])
                    Y = Y.drop(np.arange(0,n),axis=1)
            for v in np.arange(n):
                layer_accuracy.append(accuracy_score(target,X[v]))
                print(f'\n Accuracy Score of Layer-{L} Model-{v} = {layer_accuracy[v]}')
                print('--------------------------------------------------\n')    
            if self.s[L] == 1:
                data = pd.concat([data,X],axis=1)
            elif self.s[L] == 0:
                data = X.copy()
            L+=1
            print(f'\n Layer {L} Starts Here: --{L}--{L}--{L}--{L}--{L}--{L}--{L}--{L}--{L}--- \n')
    
    def predict(self,data):
        pred = pd.DataFrame()
        for model in self.models:
            for n in np.arange(len(model)): 
                pred[n] = model[n].predict(data)
            data = pred.copy()
            pred = pd.DataFrame()
        return data

In [29]:
%macro -q __SNKFC 28

In [30]:
%store __SNKFC

Stored '__SNKFC' (Macro)


In [49]:
class SNKFR:
    
    def __init__(self, models, k,s):
        self.models = models
        self.k = k
        self.s = s
    
    def fit(self,data,target):
        kf = KFold(n_splits=self.k)
        n1 = len(self.models)
        L=0
        for model in self.models:
            n = len(model)
            i=0
            rmse = []
            pred = pd.DataFrame()
            for train_index , val_index in kf.split(data):
                X_train , X_val = data.loc[train_index],data.loc[val_index]
                y_train , y_val = target[train_index],target[val_index]
                c= 0
                while c < n:
                    model[c].fit(X_train,y_train)
                    pred[i] = model[c].predict(X_val)
                    c=c+1
                    i=i+1
            for t in np.arange(kf.n_splits):
                if t == 0:
                    X = pred[np.arange(0,n)]
                else:
                    Y = pred[np.arange(t*n,(1+t)*n)].set_index(np.arange(t*int(data.shape[0]/kf.n_splits),(t+1)*int(data.shape[0]/kf.n_splits)))
                    for g in np.arange(n):
                        Y[g]=Y[(n*t)+g]
                        Y = Y.drop(n*t+g,axis=1)
                    X = pd.concat([X,Y])
                    Y = Y.drop(np.arange(0,n),axis=1)
            for v in np.arange(n):
                rmse.append(np.sqrt(mean_squared_error(target,X[v])))
                print(f'\n RMS Error of Layer-{L} Model-{v} = {rmse[v]}')
                print('--------------------------------------------------\n')     
            if L < len(self.s):
                if self.s[L] == 1 :
                    data = pd.concat([data,X],axis=1)
                elif self.s[L] == 0:
                    data = X.copy()
                print(f'\n Layer {L} Ends Here: --{L}--{L}--{L}--{L}--{L}--{L}--{L}--{L}--{L}--- \n')    
            L+=1
    
    def predict(self,data):
        pred = pd.DataFrame()
        L=0
        for model in self.models:
            for n in np.arange(len(model)): 
                pred[n] = model[n].predict(data) 
            if L == len(self.s):
                return pred
            else:
                if self.s[L] == 1 :
                    data = pd.concat([data,pred],axis=1)
                elif self.s[L] == 0:
                    data = pred.copy()
                pred = pd.DataFrame()
            L+=1

In [50]:
%macro -q __SNKFR 49

In [51]:
%store __SNKFR

Stored '__SNKFR' (Macro)
